# Bias detection

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

In [ ]:
# response: biased or non-biased

def bias_det(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        You will be provided with a text, and your task is to classify it into 
        one of two categories: "biased" or "non-biased." A text is considered "biased" 
        if it expresses a personal opinion, uses emotional or partial language, or promotes 
        a particular perspective at the expense of others. A text is considered "non-biased" 
        if it presents facts in a neutral and objective manner, without evident influences.
        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS.
        Below, I will show you some classification examples to help you understand 
        how to distinguish between the two types of texts. \n

        Example 1: \n
        Text: "The new educational program is a complete failure, designed with no regard for the needs of the students." \n
        Answer: biased \n

        Example 2: \n
        Text: "The new educational program was introduced with the goal of improving student performance in scientific subjects." \n
        Answer: non-biased \n

        Example 3: \n
        Text: "The current government has done a terrible job managing the economic crisis, causing more harm than good." \n
        Answer: biased \n

        Example 4: \n
        Text: "The government has implemented a series of measures to address the economic crisis, with mixed results according to various analysts." \n
        Answer: non-biased \n
   
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        Text: {text} \n
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    bias_det_chain = prompt | llm | StrOutputParser()
    return bias_det_chain

# Debiasing

In [ ]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes.
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    debiasing = prompt | llm | StrOutputParser()
    return debiasing